In [9]:
pip install pandas DateTime plotly dash matplotlib dash_bootstrap_components dash_bootstrap_templates

In [12]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import matplotlib as mpl
import matplotlib.pyplot as plt
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

In [ ]:
## Create a dash application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])
load_figure_template('SOLAR')

## Read data files
df_profile = pd.read_excel('profile.xlsx', sheet_name = 'EOS')
df_preference = pd.read_excel('profile.xlsx', sheet_name = 'Preference')

## Helper function that outputs the best attributes of a Naval Officer, based on his / her NRIC
def get_top_attributes(nric):
    
    top_attributes = []
    attributes = ['Intellect', 'Affability', 'Mission_Ability', 'Developmental_Ability']
    try: 
        max_value = df_profile[df_profile['NRIC'] == nric][attributes].max(axis = 1).values[0]
    except:
        print("No such Naval Officer")
    for i in list(range(4)):
        if df_profile[df_profile['NRIC'] == nric][attributes[i]].values[0] == max_value:
            top_attributes.append(attributes[i])
    
    return top_attributes

## Helper function that outputs the career tracks suitable for a Naval Officer, based on his / her best attributes
def get_track(top_attributes):
    tracks = []
    for attribute in top_attributes:
        track = df_preference[df_preference['Attribute'] == attribute]['Track']
        tracks.append(track.values[0])
    return tracks

## Helper function that outputs the corresponding appointment, given a career track
def get_appointment(top_tracks, nric):
    first = df_profile[df_profile['NRIC'] == nric]['First_Choice'].values[0]
    second = df_profile[df_profile['NRIC'] == nric]['Second_Choice'].values[0]
    third = df_profile[df_profile['NRIC'] == nric]['Third_Choice'].values[0]
    fourth = df_profile[df_profile['NRIC'] == nric]['Fourth_Choice'].values[0]
    if first in top_tracks:
        return df_preference[df_preference['Track'] == first]['Appointment'].values[0]
    elif second in top_tracks:
        return df_preference[df_preference['Track'] == second]['Appointment'].values[0]
    elif third in top_tracks:
        return df_preference[df_preference['Track'] == third]['Appointment'].values[0]
    else: 
        return df_preference[df_preference['Track'] == fourth]['Appointment'].values[0]

## Helper function that outputs appointments suitable for a serviceman / servicewoman


## Application layout
app.layout = html.Div(children=[html.H1('Career Paths Profile',
                                        style={'textAlign': 'center', 'color': '#E3CF57','font-size': 40}),
                                html.Div("""This dashboard is a prototype and does not include any real-life data. 
                                            In the NRIC input box below, try any of the following:
                                            S0000001A, S0000002B, S0000003C, S0000004D, S0000005E
                                         """, style={'font-size': 20}),
                                html.Br(),
                                html.Div(["NRIC: ", dcc.Input(id='nric', value='S0000005E', 
                                type='text', style={'height':'50px', 'font-size': 20}),], 
                                style={'font-size': 20}),
                                html.Br(),
                                html.Div(dcc.Graph(id='radar-plot')),
                                html.Br(),
                                html.Br(),
                                html.Div(id='recommended-goal', style={'color': 'red', 'font-size': 30}),
                                html.Br(),
                                html.Div("""In the NRIC input box below, try any of the following:
                                            HNP, HNI, HNT, FC
                                         """, style={'font-size': 20}),
                                html.Br(),
                                html.Div(["Career End Goal: ", dcc.Input(id='career-end-goal', value='HNP', 
                                type='text', style={'height':'50px', 'font-size': 20}),], 
                                style={'font-size': 20}),
                                html.Br(),
                                html.Br(),
                                html.Div(dcc.Graph(id='line-plot')),
                                html.Br(),
                                html.Br()
                                ])

## Callback function to output a Naval Officer's recommended career goal
@app.callback(Output(component_id = 'recommended-goal', component_property = 'children'),
              Input(component_id = 'nric', component_property = 'value'))
def get_recommended_end_goal(nric):
    top_attributes = get_top_attributes(nric)
    top_tracks = get_track(top_attributes)
    return "Recommended Career End Goal: ", get_appointment(top_tracks, nric)

## Callback function to output radar plot showing a Naval Officer's attributes 
## and a line plot showing the path to reach the recommended career end goal
@app.callback([Output(component_id='line-plot', component_property='figure'),
               Output(component_id='radar-plot', component_property='figure')],
              Input(component_id='nric', component_property='value'),
              Input(component_id='career-end-goal', component_property='value'))
def get_graph(nric, appointment):
    
    # Radar chart for profile attributes
    
    attributes = ['Intellect', 'Affability', 'Mission_Ability', 'Developmental_Ability', 'Self_Awareness']
    try:
        data = df_profile[df_profile['NRIC'] == nric]
        values = data[attributes].values[0]
    except:
        print('No such Naval Officer')
    
    radar_fig = px.line_polar(data, r = values, theta = attributes, line_close = True)
    radar_fig.update_layout(
        title = data['Rank'].values[0] + " " + data['Name'].values[0] + " Attributes",
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 5]
        )),
    )
    
    # Line graph for career path visualisation
    try: 
        df_path = pd.read_excel('paths.xlsx', sheet_name = appointment)
    except:
        print("Input is not a valid appointment")
    
    line_fig = px.line(df_path, x = 'Year', y = ['Path1', 'Path2'], markers = True)

    line_fig.update_layout(
        title = 'Career Path to ' + appointment,
        xaxis_title = 'Year',
        yaxis_title = 'Department',
        legend_title = 'Path',
        yaxis = dict(
            tickmode = 'array',
            tickvals = list(range(1,9)),
            ticktext = ['NPD', 'NID', 'NOD', 'NTD', 'NPLD', 'NTD', 'Fleet', 'MSTF']
        ),
    )
    return [line_fig, radar_fig]

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jan/2023 10:02:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 10:02:14] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 10:02:14] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 10:02:14] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 10:02:14] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 10:02:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2023 10:02:14] "POST /_dash-update-component HTTP/1.1" 200 -
